In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Evaluation on Test Set
-----------------------------
Load and evaluate a single TFLite model on the test set
"""

import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "left_eye_1_single_eye_resnet18.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# LOAD TEST DATA
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

dirs = {
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

test_imgs, test_lbls, test_filenames = [], [], []
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# 🔀 Randomize the test set (reproducibly)
print("🔀 Randomizing test dataset order...")
seed = 42
rng = np.random.default_rng(seed)
indices = rng.permutation(len(test_imgs))
test_imgs = [test_imgs[i] for i in indices]
test_lbls = [test_lbls[i] for i in indices]
test_filenames = [test_filenames[i] for i in indices]

# =========================
# TFLITE EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    print("\n🔍 Loading TFLite model and evaluating on test set...")
    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found at {TFLITE_PATH}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(TFLITE_PATH, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS:")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "left_eye_1_detailed_predictions_tflite.csv")
        )

        print(f"✅ Evaluation complete. Results saved to: {OUTPUT_DIR}")
    else:
        print("❌ TFLite evaluation failed.")

TEST: 44 anemic / 358 total
🔀 Randomizing test dataset order...

🔍 Loading TFLite model and evaluating on test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS:
Precision: 1.0000
Recall:    0.9773
F1 score:  0.9885
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 43, 314, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/left_eye_1_detailed_predictions_tflite.csv
✅ Evaluation complete. Results saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
import os
import json
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
RESOLUTION = 224  # Add this — needed for TFLite preprocessing

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

os.environ["PYTHONHASHSEED"] = str(SEED)
set_global_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "hexa_eye_resnet18_shared.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
output_dir = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

test_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_hexa(dirs_map):
    suffixes = {
        'left1': '_left_eye_1.png', 'left2': '_left_eye_2.png', 'left3': '_left_eye_3.png',
        'right1': '_right_eye_1.png', 'right2': '_right_eye_2.png', 'right3': '_right_eye_3.png'
    }
    bases_sets = []
    for k in dirs_map.keys():
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    inter = set.intersection(*bases_sets)
    return sorted(inter)

def load_hexa_images_by_bases_with_filenames(dirs_map, bases):
    out = {f'r{i}': [] for i in range(1,7)}
    out['filenames'] = []
    key_map = [
        ('left1', '_left_eye_1.png'),
        ('left2', '_left_eye_2.png'),
        ('left3', '_left_eye_3.png'),
        ('right1', '_right_eye_1.png'),
        ('right2', '_right_eye_2.png'),
        ('right3', '_right_eye_3.png')
    ]
    for b in bases:
        imgs, failed = [], False
        for long_k, suf in key_map:
            path = os.path.join(dirs_map[long_k], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not failed:
            for i, img in enumerate(imgs):
                out[f'r{i+1}'].append(img)
            out['filenames'].append(b)
    return out

def prepare_dataset_hexa_with_filenames(anemic_dirs, non_dirs, split_name="(split)"):
    bases_a = common_bases_hexa(anemic_dirs)
    bases_n = common_bases_hexa(non_dirs)
    imgs_a = load_hexa_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_hexa_images_by_bases_with_filenames(non_dirs, bases_n)

    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'r4': imgs_a['r4'] + imgs_n['r4'],
        'r5': imgs_a['r5'] + imgs_n['r5'],
        'r6': imgs_a['r6'] + imgs_n['r6'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }
    print(f"✅ {split_name}: anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    try:
        df_bases = pd.DataFrame({'class': ['anemic']*len(bases_a) + ['non_anemic']*len(bases_n),
                                 'base_id': bases_a + bases_n})
        df_bases.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_hexa.csv"), index=False)
    except Exception:
        pass
    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in dirs_map.keys():
        p = dirs_map[k]; c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# LOAD DATA
# =========================
print_dir_stats("TEST  anemic (HEXA)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (HEXA)", test_dirs_non)

test_data = prepare_dataset_hexa_with_filenames(test_dirs_anemic, test_dirs_non, split_name="TEST")

# =========================
# TFLITE RE-EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution):
    import tensorflow as tf
    from PIL import Image
    import numpy as np
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Sort inputs by name to ensure consistent order (r1, r2, ..., r6)
    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 TFLite input names: {[d['name'] for d in sorted_inputs]}")

    # Assume all inputs have same shape
    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) == 4:
        if first_shape[-1] == 3:  # NHWC
            layout = 'NHWC'
            resize_h, resize_w = int(first_shape[1]), int(first_shape[2])
        elif first_shape[1] == 3:  # NCHW
            layout = 'NCHW'
            resize_h, resize_w = int(first_shape[2]), int(first_shape[3])
        else:
            raise ValueError(f"Cannot infer layout from shape: {first_shape}")
    else:
        raise ValueError(f"Expected 4D input, got {first_shape}")

    print(f"   ➤ Detected layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        img_pil = Image.fromarray(img_rgb)
        img_resized = img_pil.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(img_resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, all_filenames = [], [], [], test_data['filenames']

    total = len(test_data['label'])
    for i in range(total):
        imgs = [test_data[f'r{j}'][i] for j in range(1,7)]
        label = test_data['label'][i]

        # Set each input tensor
        for idx, detail in enumerate(sorted_inputs):
            raw_img = imgs[idx]
            processed = preprocess_pil_style(raw_img)

            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:  # NHWC
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])

            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logit = float(np.array(output).reshape(-1)[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    if len(set(all_labels)) < 2:
        auc = float('nan')
    else:
        auc = roc_auc_score(all_labels, all_probs)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds


# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")


# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc

    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    print("\n🔍 Loading TFLite model and evaluating on HEXA test set...")
    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found at {TFLITE_PATH}")

    # ✅ Fix: Pass test_data (not undefined variables)
    tflite_metrics = evaluate_tflite_model_with_predictions(TFLITE_PATH, test_data, RESOLUTION)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (HEXA-EYE):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save CSV
        csv_path = os.path.join(OUTPUT_DIR, "hexa_eye_detailed_predictions_tflite.csv")
        save_predictions_to_csv(tflite_filenames, tflite_labels, tflite_preds, tflite_probs, csv_path)

        # Plot results
        plot_roc_curve(
            tflite_labels, tflite_probs,
            "ROC Curve - TFLite Hexa-Eye Model",
            os.path.join(OUTPUT_DIR, "hexa_eye_roc_curve_tflite.png")
        )
        plot_confusion_matrix(
            tflite_labels, tflite_preds,
            "Confusion Matrix - TFLite Hexa-Eye Model",
            os.path.join(OUTPUT_DIR, "hexa_eye_confusion_matrix_tflite.png")
        )

        print(f"✅ Evaluation complete. All outputs saved to: {OUTPUT_DIR}")
    else:
        print("❌ TFLite evaluation failed.")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TEST  anemic (HEXA)
left1   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=44
left2   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=42
left3   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=40
right3  | /home/u

2025-11-28 22:24:19.959391: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-28 22:24:19.960626: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...



📊 TFLITE TEST RESULTS (HEXA-EYE):
Precision: 1.0000
Recall:    0.9643
F1 score:  0.9818
Accuracy:  0.9962
AUC:       1.0000
TP, TN, FP, FN: 27, 237, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/hexa_eye_detailed_predictions_tflite.csv
✅ Evaluation complete. All outputs saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Re-evaluation for Tri-Left-Eye Model
- Loads test data from tri_left_eye/.../anemic_test_roi & anemic_not_test_roi
- Evaluates exported .tflite model
- Saves CSV + optional plots
"""

import os
import cv2
import pandas as pd
import numpy as np
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
from sklearn.metrics import (precision_recall_fscore_support, accuracy_score,
                             roc_auc_score, confusion_matrix, roc_curve)
import tensorflow as tf

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "tri_left_eye_resnet18_shared.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Test directories
test_dirs_anemic = {
    'left1': os.path.join(BASE_PATH, 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'),
    'left2': os.path.join(BASE_PATH, 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'),
    'left3': os.path.join(BASE_PATH, 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'),
}
test_dirs_non = {
    'left1': os.path.join(BASE_PATH, 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'),
    'left2': os.path.join(BASE_PATH, 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'),
    'left3': os.path.join(BASE_PATH, 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'),
}

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_left(dirs_map):
    suffixes = {'left1': '_left_eye_1.png', 'left2': '_left_eye_2.png', 'left3': '_left_eye_3.png'}
    bases_sets = []
    for k in ['left1', 'left2', 'left3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    return sorted(set.intersection(*bases_sets))

def load_tri_images_by_bases_with_filenames(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': [], 'filenames': []}
    key_map = [('r1', 'left1', '_left_eye_1.png'),
               ('r2', 'left2', '_left_eye_2.png'),
               ('r3', 'left3', '_left_eye_3.png')]
    for b in bases:
        imgs, failed = {}, False
        for out_key, dir_key, suf in key_map:
            path = os.path.join(dirs_map[dir_key], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[out_key] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if not failed:
            out['r1'].append(imgs['r1'])
            out['r2'].append(imgs['r2'])
            out['r3'].append(imgs['r3'])
            out['filenames'].append(b)
    return out

def prepare_test_data(anemic_dirs, non_dirs):
    bases_a = common_bases_left(anemic_dirs)
    bases_n = common_bases_left(non_dirs)
    imgs_a = load_tri_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_tri_images_by_bases_with_filenames(non_dirs, bases_n)
    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }
    print(f"✅ Loaded {len(data['label'])} test samples "
          f"({len(imgs_a['r1'])} anemic, {len(imgs_n['r1'])} non-anemic)")
    return data

# =========================
# TFLITE EVALUATION
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Sort by name to ensure r1→input1, r2→input2, r3→input3
    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 Input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) != 4:
        raise ValueError(f"Expected 4D input, got {first_shape}")

    # Auto-detect layout
    if first_shape[1] == 3 and first_shape[3] != 3:
        layout = 'NCHW'
        resize_h, resize_w = int(first_shape[2]), int(first_shape[3])
    elif first_shape[3] == 3 and first_shape[1] != 3:
        layout = 'NHWC'
        resize_h, resize_w = int(first_shape[1]), int(first_shape[2])
    else:
        if first_shape[-1] == 3:
            layout = 'NHWC'
            resize_h, resize_w = int(first_shape[1]), int(first_shape[2])
        elif first_shape[1] == 3:
            layout = 'NCHW'
            resize_h, resize_w = int(first_shape[2]), int(first_shape[3])
        else:
            raise ValueError(f"Cannot infer layout from {first_shape}")

    print(f"   ➤ Layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        pil_img = Image.fromarray(img_rgb)
        resized = pil_img.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data['r1'][i], test_data['r2'][i], test_data['r3'][i]]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            processed = preprocess_pil_style(imgs[idx])
            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])
            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        logit = float(interpreter.get_tensor(output_details[0]['index']).flatten()[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    # Metrics
    if len(set(all_labels)) < 2:
        p = r = f1 = acc = auc = float('nan')
        tn = fp = fn = tp = 0
    else:
        p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
        acc = accuracy_score(all_labels, all_preds)
        auc = roc_auc_score(all_labels, all_probs)
        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0, 1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, filenames, all_preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    true_str = ['Anemic' if y == 1 else 'Non-Anemic' for y in true_labels]
    pred_str = ['Anemic' if y == 1 else 'Non-Anemic' for y in pred_labels]
    tp = [1 if t == 1 and p == 1 else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if t == 0 and p == 0 else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if t == 0 and p == 1 else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if t == 1 and p == 0 else 0 for t, p in zip(true_labels, pred_labels)]

    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_str,
        'predicted_value': pred_str,
        'predicted_probability': pred_probs,
        'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn
    })
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# PLOTTING (Optional)
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = roc_auc_score(y_true, y_scores)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(title); plt.legend(loc="lower right"); plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight'); plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title); plt.ylabel('True Label'); plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight'); plt.close()

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    print("🔍 Loading test data...")
    test_data = prepare_test_data(test_dirs_anemic, test_dirs_non)

    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found: {TFLITE_PATH}")

    print("🔍 Evaluating TFLite model...")
    metrics = evaluate_tflite_model_with_predictions(TFLITE_PATH, test_data)

    if metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, labels, probs, filenames, preds = metrics

        print("\n📊 TFLite Results (Tri-Left-Eye):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 Score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP/TN/FP/FN: {tp}/{tn}/{fp}/{fn}")

        # Save CSV
        csv_path = os.path.join(OUTPUT_DIR, "tri_left_tflite_predictions.csv")
        save_predictions_to_csv(filenames, labels, preds, probs, csv_path)

        # Optional: save plots
        plot_roc_curve(labels, probs,
                       "ROC Curve - TFLite (Tri-Left-Eye)",
                       os.path.join(OUTPUT_DIR, "tri_left_roc_tflite.png"))
        plot_confusion_matrix(labels, preds,
                              "Confusion Matrix - TFLite (Tri-Left-Eye)",
                              os.path.join(OUTPUT_DIR, "tri_left_cm_tflite.png"))

        print(f"\n✅ Evaluation complete. Outputs saved to: {OUTPUT_DIR}")
    else:
        print("❌ Evaluation failed.")

🔍 Loading test data...
✅ Loaded 305 test samples (35 anemic, 270 non-anemic)
🔍 Evaluating TFLite model...
🔍 Input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0']
   ➤ Layout: NCHW, size: 224x224


2025-11-28 22:25:41.705141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-28 22:25:41.706385: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...



📊 TFLite Results (Tri-Left-Eye):
Precision: 1.0000
Recall:    0.9429
F1 Score:  0.9706
Accuracy:  0.9934
AUC:       0.9996
TP/TN/FP/FN: 33/270/0/2
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/tri_left_tflite_predictions.csv

✅ Evaluation complete. Outputs saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Re-evaluation for Tri-right-Eye Model
- Loads test data from tri_right_eye/.../anemic_test_roi & anemic_not_test_roi
- Evaluates exported .tflite model
- Saves CSV + optional plots
"""

import os
import cv2
import pandas as pd
import numpy as np
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
from sklearn.metrics import (precision_recall_fscore_support, accuracy_score,
                             roc_auc_score, confusion_matrix, roc_curve)
import tensorflow as tf

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "tri_right_eye_resnet18_shared.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Test directories
test_dirs_anemic = {
    'right1': os.path.join(BASE_PATH, 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'),
    'right2': os.path.join(BASE_PATH, 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'),
    'right3': os.path.join(BASE_PATH, 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'),
}
test_dirs_non = {
    'right1': os.path.join(BASE_PATH, 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'),
    'right2': os.path.join(BASE_PATH, 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'),
    'right3': os.path.join(BASE_PATH, 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'),
}

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_right(dirs_map):
    suffixes = {'right1': '_right_eye_1.png', 'right2': '_right_eye_2.png', 'right3': '_right_eye_3.png'}
    bases_sets = []
    for k in ['right1', 'right2', 'right3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    return sorted(set.intersection(*bases_sets))

def load_tri_images_by_bases_with_filenames(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': [], 'filenames': []}
    key_map = [('r1', 'right1', '_right_eye_1.png'),
               ('r2', 'right2', '_right_eye_2.png'),
               ('r3', 'right3', '_right_eye_3.png')]
    for b in bases:
        imgs, failed = {}, False
        for out_key, dir_key, suf in key_map:
            path = os.path.join(dirs_map[dir_key], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[out_key] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if not failed:
            out['r1'].append(imgs['r1'])
            out['r2'].append(imgs['r2'])
            out['r3'].append(imgs['r3'])
            out['filenames'].append(b)
    return out

def prepare_test_data(anemic_dirs, non_dirs):
    bases_a = common_bases_right(anemic_dirs)
    bases_n = common_bases_right(non_dirs)
    imgs_a = load_tri_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_tri_images_by_bases_with_filenames(non_dirs, bases_n)
    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }
    print(f"✅ Loaded {len(data['label'])} test samples "
          f"({len(imgs_a['r1'])} anemic, {len(imgs_n['r1'])} non-anemic)")
    return data

# =========================
# TFLITE EVALUATION
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Sort by name to ensure r1→input1, r2→input2, r3→input3
    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 Input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) != 4:
        raise ValueError(f"Expected 4D input, got {first_shape}")

    # Auto-detect layout
    if first_shape[1] == 3 and first_shape[3] != 3:
        layout = 'NCHW'
        resize_h, resize_w = int(first_shape[2]), int(first_shape[3])
    elif first_shape[3] == 3 and first_shape[1] != 3:
        layout = 'NHWC'
        resize_h, resize_w = int(first_shape[1]), int(first_shape[2])
    else:
        if first_shape[-1] == 3:
            layout = 'NHWC'
            resize_h, resize_w = int(first_shape[1]), int(first_shape[2])
        elif first_shape[1] == 3:
            layout = 'NCHW'
            resize_h, resize_w = int(first_shape[2]), int(first_shape[3])
        else:
            raise ValueError(f"Cannot infer layout from {first_shape}")

    print(f"   ➤ Layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        pil_img = Image.fromarray(img_rgb)
        resized = pil_img.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data['r1'][i], test_data['r2'][i], test_data['r3'][i]]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            processed = preprocess_pil_style(imgs[idx])
            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])
            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        logit = float(interpreter.get_tensor(output_details[0]['index']).flatten()[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    # Metrics
    if len(set(all_labels)) < 2:
        p = r = f1 = acc = auc = float('nan')
        tn = fp = fn = tp = 0
    else:
        p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
        acc = accuracy_score(all_labels, all_preds)
        auc = roc_auc_score(all_labels, all_probs)
        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0, 1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, filenames, all_preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    true_str = ['Anemic' if y == 1 else 'Non-Anemic' for y in true_labels]
    pred_str = ['Anemic' if y == 1 else 'Non-Anemic' for y in pred_labels]
    tp = [1 if t == 1 and p == 1 else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if t == 0 and p == 0 else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if t == 0 and p == 1 else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if t == 1 and p == 0 else 0 for t, p in zip(true_labels, pred_labels)]

    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_str,
        'predicted_value': pred_str,
        'predicted_probability': pred_probs,
        'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn
    })
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# PLOTTING (Optional)
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = roc_auc_score(y_true, y_scores)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(title); plt.legend(loc="lower right"); plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight'); plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title); plt.ylabel('True Label'); plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight'); plt.close()

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    print("🔍 Loading test data...")
    test_data = prepare_test_data(test_dirs_anemic, test_dirs_non)

    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found: {TFLITE_PATH}")

    print("🔍 Evaluating TFLite model...")
    metrics = evaluate_tflite_model_with_predictions(TFLITE_PATH, test_data)

    if metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, labels, probs, filenames, preds = metrics

        print("\n📊 TFLite Results (Tri-right-Eye):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 Score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP/TN/FP/FN: {tp}/{tn}/{fp}/{fn}")

        # Save CSV
        csv_path = os.path.join(OUTPUT_DIR, "tri_right_tflite_predictions.csv")
        save_predictions_to_csv(filenames, labels, preds, probs, csv_path)

        # Optional: save plots
        plot_roc_curve(labels, probs,
                       "ROC Curve - TFLite (Tri-right-Eye)",
                       os.path.join(OUTPUT_DIR, "tri_right_roc_tflite.png"))
        plot_confusion_matrix(labels, preds,
                              "Confusion Matrix - TFLite (Tri-right-Eye)",
                              os.path.join(OUTPUT_DIR, "tri_right_cm_tflite.png"))

        print(f"\n✅ Evaluation complete. Outputs saved to: {OUTPUT_DIR}")
    else:
        print("❌ Evaluation failed.")

🔍 Loading test data...
✅ Loaded 318 test samples (34 anemic, 284 non-anemic)
🔍 Evaluating TFLite model...
🔍 Input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0']
   ➤ Layout: NCHW, size: 224x224


2025-11-28 22:26:29.810386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-28 22:26:29.811609: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...



📊 TFLite Results (Tri-right-Eye):
Precision: 1.0000
Recall:    0.9118
F1 Score:  0.9538
Accuracy:  0.9906
AUC:       1.0000
TP/TN/FP/FN: 31/284/0/3
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/tri_right_tflite_predictions.csv

✅ Evaluation complete. Outputs saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Evaluation on Test Set
-----------------------------
Load and evaluate a single TFLite model on the test set
"""

import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "left_eye_2_single_eye_resnet18.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# LOAD TEST DATA
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

dirs = {
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

test_imgs, test_lbls, test_filenames = [], [], []
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# 🔀 Randomize the test set (reproducibly)
print("🔀 Randomizing test dataset order...")
seed = 42
rng = np.random.default_rng(seed)
indices = rng.permutation(len(test_imgs))
test_imgs = [test_imgs[i] for i in indices]
test_lbls = [test_lbls[i] for i in indices]
test_filenames = [test_filenames[i] for i in indices]

# =========================
# TFLITE EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    print("\n🔍 Loading TFLite model and evaluating on test set...")
    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found at {TFLITE_PATH}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(TFLITE_PATH, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS:")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "left_eye_2_detailed_predictions_tflite.csv")
        )

        print(f"✅ Evaluation complete. Results saved to: {OUTPUT_DIR}")
    else:
        print("❌ TFLite evaluation failed.")

TEST: 42 anemic / 361 total
🔀 Randomizing test dataset order...

🔍 Loading TFLite model and evaluating on test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS:
Precision: 1.0000
Recall:    0.9762
F1 score:  0.9880
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 41, 319, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/left_eye_2_detailed_predictions_tflite.csv
✅ Evaluation complete. Results saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Evaluation on Test Set
-----------------------------
Load and evaluate a single TFLite model on the test set
"""

import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "left_eye_3_single_eye_resnet18.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# LOAD TEST DATA
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

dirs = {
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

test_imgs, test_lbls, test_filenames = [], [], []
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# 🔀 Randomize the test set (reproducibly)
print("🔀 Randomizing test dataset order...")
seed = 42
rng = np.random.default_rng(seed)
indices = rng.permutation(len(test_imgs))
test_imgs = [test_imgs[i] for i in indices]
test_lbls = [test_lbls[i] for i in indices]
test_filenames = [test_filenames[i] for i in indices]

# =========================
# TFLITE EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    print("\n🔍 Loading TFLite model and evaluating on test set...")
    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found at {TFLITE_PATH}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(TFLITE_PATH, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS:")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "left_eye_3_detailed_predictions_tflite.csv")
        )

        print(f"✅ Evaluation complete. Results saved to: {OUTPUT_DIR}")
    else:
        print("❌ TFLite evaluation failed.")

TEST: 43 anemic / 361 total
🔀 Randomizing test dataset order...

🔍 Loading TFLite model and evaluating on test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS:
Precision: 1.0000
Recall:    0.9767
F1 score:  0.9882
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 42, 318, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/left_eye_3_detailed_predictions_tflite.csv
✅ Evaluation complete. Results saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Evaluation on Test Set
-----------------------------
Load and evaluate a single TFLite model on the test set
"""

import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "right_eye_1_single_eye_resnet18.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# LOAD TEST DATA
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

dirs = {
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

test_imgs, test_lbls, test_filenames = [], [], []
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# 🔀 Randomize the test set (reproducibly)
print("🔀 Randomizing test dataset order...")
seed = 42
rng = np.random.default_rng(seed)
indices = rng.permutation(len(test_imgs))
test_imgs = [test_imgs[i] for i in indices]
test_lbls = [test_lbls[i] for i in indices]
test_filenames = [test_filenames[i] for i in indices]

# =========================
# TFLITE EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    print("\n🔍 Loading TFLite model and evaluating on test set...")
    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found at {TFLITE_PATH}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(TFLITE_PATH, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS:")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "right_eye_1_detailed_predictions_tflite.csv")
        )

        print(f"✅ Evaluation complete. Results saved to: {OUTPUT_DIR}")
    else:
        print("❌ TFLite evaluation failed.")

TEST: 43 anemic / 365 total
🔀 Randomizing test dataset order...

🔍 Loading TFLite model and evaluating on test set...
🔍 TFLite model input shape: [   1    3 1024 1024]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS:
Precision: 0.6500
Recall:    0.3023
F1 score:  0.4127
Accuracy:  0.8986
AUC:       0.8706
TP, TN, FP, FN: 13, 315, 7, 30
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/right_eye_1_detailed_predictions_tflite.csv
✅ Evaluation complete. Results saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Evaluation on Test Set
-----------------------------
Load and evaluate a single TFLite model on the test set
"""

import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "right_eye_2_single_eye_resnet18.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# LOAD TEST DATA
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

dirs = {
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

test_imgs, test_lbls, test_filenames = [], [], []
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# 🔀 Randomize the test set (reproducibly)
print("🔀 Randomizing test dataset order...")
seed = 42
rng = np.random.default_rng(seed)
indices = rng.permutation(len(test_imgs))
test_imgs = [test_imgs[i] for i in indices]
test_lbls = [test_lbls[i] for i in indices]
test_filenames = [test_filenames[i] for i in indices]

# =========================
# TFLITE EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    print("\n🔍 Loading TFLite model and evaluating on test set...")
    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found at {TFLITE_PATH}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(TFLITE_PATH, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS:")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "right_eye_2_detailed_predictions_tflite.csv")
        )

        print(f"✅ Evaluation complete. Results saved to: {OUTPUT_DIR}")
    else:
        print("❌ TFLite evaluation failed.")

TEST: 40 anemic / 369 total
🔀 Randomizing test dataset order...

🔍 Loading TFLite model and evaluating on test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS:
Precision: 1.0000
Recall:    0.9750
F1 score:  0.9873
Accuracy:  0.9973
AUC:       1.0000
TP, TN, FP, FN: 39, 329, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/right_eye_2_detailed_predictions_tflite.csv
✅ Evaluation complete. Results saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
# -*- coding: utf-8 -*-
"""
TFLite Evaluation on Test Set
-----------------------------
Load and evaluate a single TFLite model on the test set
"""

import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from torchvision.transforms.functional import to_tensor, normalize
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/"
TFLITE_PATH = os.path.join(BASE_PATH, "TFlite files", "right_eye_3_single_eye_resnet18.tflite")
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# LOAD TEST DATA
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

dirs = {
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

test_imgs, test_lbls, test_filenames = [], [], []
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# 🔀 Randomize the test set (reproducibly)
print("🔀 Randomizing test dataset order...")
seed = 42
rng = np.random.default_rng(seed)
indices = rng.permutation(len(test_imgs))
test_imgs = [test_imgs[i] for i in indices]
test_lbls = [test_lbls[i] for i in indices]
test_filenames = [test_filenames[i] for i in indices]

# =========================
# TFLITE EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")

    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce:
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]

    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]

    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })

    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    print("\n🔍 Loading TFLite model and evaluating on test set...")
    if not os.path.exists(TFLITE_PATH):
        raise FileNotFoundError(f"TFLite model not found at {TFLITE_PATH}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(TFLITE_PATH, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS:")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "right_eye_3_detailed_predictions_tflite.csv")
        )

        print(f"✅ Evaluation complete. Results saved to: {OUTPUT_DIR}")
    else:
        print("❌ TFLite evaluation failed.")

TEST: 40 anemic / 368 total
🔀 Randomizing test dataset order...

🔍 Loading TFLite model and evaluating on test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS:
Precision: 1.0000
Recall:    0.8750
F1 score:  0.9333
Accuracy:  0.9864
AUC:       0.9987
TP, TN, FP, FN: 35, 328, 0, 5
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files/right_eye_3_detailed_predictions_tflite.csv
✅ Evaluation complete. Results saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite files


In [ ]:
''' done '''

' done '

In [ ]:
import os
import shutil

BASE_PATH = BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
OUTPUT_DIR = os.path.join(BASE_PATH, "TFlite files")   # folder to zip

ZIP_PATH = os.path.join(BASE_PATH, "TFlite_files.zip")  # name of zip file

# Create ZIP
shutil.make_archive(
    base_name=ZIP_PATH.replace(".zip", ""),  # remove .zip for function
    format='zip',
    root_dir=OUTPUT_DIR
)

print("Zip created at:", ZIP_PATH)


Zip created at: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/TFlite_files.zip
